# UAS PPW

In [9]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df= pd.read_csv = '/content/drive/MyDrive/projectPPW/tugas/CrawlingBerita (2).csv'
df = pd.read_csv
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/projectPPW/tugas/CrawlingBerita (2).csv'

# Cleaning data  
Tahap membersihkan data dari noise, menghapus
karakter khusus, menghapus angka dan menghilangkan spasi yang
berlebihan. Cleaning data dengan menggunakan library re

In [10]:
!pip install nltk
!pip install Sastrawi
!pip install joblib

In [11]:
import pandas as pd
import nltk
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score
import joblib

In [12]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Missing Value

In [13]:
df.isna().sum()

AttributeError: ignored

In [ ]:
data = df.dropna() #menghapus missing value
data.reset_index(drop=True, inplace=True) #atur indeks data mulai dari 0 lagi
data

In [ ]:
total = data['Label'].value_counts()
total

In [ ]:
data.shape

In [ ]:
berita = data["Content"].astype(str)

In [ ]:
berita

## Lowecase  
proses untuk mengubah semua huruf dalam dokumen
menjadi huruf kecil (lowercase), yang bertujuan untuk memastikan
konsistensi dalam pemrosesan teks. Case folding dengan lowercase

In [ ]:
preprocessing = berita.str.lower()

In [ ]:
preprocessing

## Tokenisasi  
proses membagi teks menjadi unit-unit terpisah yang
disebut token. Token merupakan unit terkecil dalam teks, yang bisa
berupa kata, frasa, karakter, atau bahkan simbol tertentu. Tujuan dari
tokenisasi adalah untuk mempermudah pemrosesan dan analisis lebih
lanjut terhadap teks. Tokenizing dengan library nltk.tokenize.


In [ ]:
def process_tokenize(text):
    text = text.split()
    return text

In [ ]:
preprocessing = preprocessing.apply(process_tokenize)
preprocessing

## Punctuation  
Menghapus seluruh tanda baca yang ada pada konten berita Cleaning data dengan menggunakan library re

In [ ]:
def process_punctuation(tokens):
    cleaned_tokens = [re.sub(r'[.,()&=%:-]', '', token) for token in tokens]
    cleaned_tokens = [re.sub(r'\d+', '', token) for token in cleaned_tokens]

    return cleaned_tokens

In [ ]:
preprocessing = preprocessing.apply(process_punctuation)
preprocessing

## stopword  
Tahap menghapus kata-kata yang kurang
berrmakna atau tidak memiliki arti seperti kata: saya, dan, atau. Filtering
stop word dengan library nltk, dengan menggunakan kamus stopword
bahasa Indonesia.


In [ ]:
def process_stopword_token(tokens):
    stop_words = set(stopwords.words("indonesian"))
    # custom_stop_words = ['masingmasing','tiaptiap','satusatunya', 'intinya', 'seiring']
    # stop_words.update(custom_stop_words)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(filtered_tokens)

In [ ]:
preprocessing = preprocessing.apply(process_stopword_token)
preprocessing

## Stemming  
Proses untuk menghapus kata imbuhan untuk mendapatkan
kata dasarnya. Pada proses stemming dengan menggunakan
library sastrawi. PySastrawi adalah sebuah library Python yang
menyediakan fungsi-fungsi untuk melakukan pemrosesan bahasa alami
dalam bahasa Indonesia, seperti stemming (pemenggalan kata ke bentuk
dasarnya).

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
preprocessing = preprocessing.apply(lambda text:stemmer.stem(text))
preprocessing

In [ ]:
data['preprocessing_berita'] = preprocessing
data.to_csv('/content/drive/MyDrive/Colab Notebooks/preprocessing_berita.csv', index=False)

# fitur ekstraksi  



In [ ]:
csv_preprocessing = '/content/drive/MyDrive/Colab Notebooks/preprocessing_berita.csv'
datas = pd.read_csv(csv_preprocessing)
datas

In [ ]:
preprocessing_berita = data["preprocessing_berita"]

## TF-IDF  
Pada Tahap ekstraksi fitur, dilakukan dengan metode TF-IDF (Term
Frequency-Inverse Document Frequency) melibatkan perhitungan nilai TF-IDF
untuk setiap kata dalam setiap dokumen. Menghitung TF adalah menghitung
frekuensi kemunculan setiap kata dalam dokumen tertentu. Kemudian
menghitung DF adalah menghitung jumlah dokumen dalam korpus yang
mengandung kata tertentu. Lalu IDF yaitu menghitung kebalikan dari DF, dapat
dihitung dengan menggunakan rumus logaritma dari jumlah total dokumen
dibagi dengan DF untuk kata tersebut. Dan kemudian TF-IDF yaitu mengalikan
nilai TF dengan nilai IDF untuk setiap kata dalam setiap dokumen. Akan
menghasilkan representasi vektor numerik yang memperhitungkan pentingnya
kata dalam dokumen dan dalam korpus secara keseluruhan

In [ ]:
tfidfvectorizer = TfidfVectorizer(analyzer='word')
tfidf = tfidfvectorizer.fit_transform(preprocessing_berita)
tfidf_token = tfidfvectorizer.get_feature_names_out()

In [ ]:
tfidf_df = pd.DataFrame(data = tfidf.toarray(), columns = tfidf_token)
tfidf_df

# Split Data  
Pembagian dataset dibagi menjadi dua yaitu data training dan data testing. Data training digunakan untuk proses pelatihan. Data testing digunakan untuk testing

In [ ]:
X = tfidf_df
y = data['Label']

y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

# Classification  
Metode Naive Bayes adalah metode klasifikasi yang berdasarkan pada Teorema
Bayes dengan asumsi independensi antara fitur-fitur yang ada. Metode ini cocok untuk digunakan pada data yang memiliki banyak fitur. Pada dasarnya,
algoritma Naive Bayes menghitung probabilitas posterior dari setiap kelas berdasarkan fitur-fitur yang diamati.

In [ ]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)
y_pred_naivebayes = naive_bayes_classifier.predict(X_test)

In [ ]:
cm_naivebayes = confusion_matrix(y_test, y_pred_naivebayes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_naivebayes)
accuracy = accuracy_score(y_test, y_pred_naivebayes)

plt.figure(figsize=(4, 3))
sns.heatmap(cm_naivebayes, annot=True, fmt='d', cmap='Blues')

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix for')
plt.show()

print('Accuracy =', accuracy)

In [ ]:
joblib.dump(naive_bayes_classifier, 'nb_model')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Inisialisasi model Naive Bayes
rf = RandomForestClassifier()

# Latih model pada kolom topik tertentu
rf.fit(X_train, y_train)

# Lakukan prediksi
y_pred = rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred)
print(f'Akurasi model Random Forest: {accuracy_rf}')

In [ ]:
joblib.dump(tfidfvectorizer, 'tfidf_vectorizer')

In [ ]:
joblib.dump(rf, 'random_forest_model')

# Aplikasi Predikisi

Aplikasi untuk klasifikasi berita 👉 [Streamlit](https://uasppw.streamlit.app/)